Calculation of Geometric Median, EucDist, CosDist

In [ ]:
import xarray as xr
import numpy as np
from geomedian import geometric_median, spectral_angle,medoid #import the geomedian module
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# example of calculating the geometric median  from netcdf inputs 
file = '/g/data/xc0/project/Burn_Mapping/Sumac_forest_TAS_2014_2017_100m_landsat8.nc'
data = xr.open_dataset(file)

In [3]:
def Eucdistance(ref,obs):
    EucDist = np.transpose(np.transpose(obs) - ref)
    EucNorm = np.sqrt(np.sum(EucDist**2,axis=0))
    
    return EucNorm
    
    

In [4]:
def Cosdistance(ref,obs):
    cosdist = np.empty((obs.shape[1],))
    cosdist.fill(np.nan)
    index = np.where(~np.isnan(obs[0,:]))[0]
    tmp = [1 - np.sum(ref*obs[:,t])/(np.sqrt(np.sum(ref**2))*np.sqrt(np.sum(obs[:,t]**2))) for t in index]
    cosdist[index] = np.transpose(tmp)
    return cosdist

In [ ]:
MaxInter = 60
tol      = 1.e-7
Nbands = 6
GeoMed= np.empty((Nbands,len(data.y),len(data.x)))
GeoMed.fill(np.nan)
SA = np.empty((len(data.time),len(data.y),len(data.x)))
cosdist = np.empty((len(data.time),len(data.y),len(data.x)))
Eucdist = np.empty((len(data.time),len(data.y),len(data.x)))
#for x in range(0,len(data.x)):
for y in range(0,len(data.y)):
    X = np.empty((Nbands,len(data.time),len(data.x)))
    X[0,:,:] = data.blue[:,y,:]*data.pixmask[:,y,:] 
    X[1,:,:] = data.green[:,y,:]*data.pixmask[:,y,:]
    X[2,:,:] = data.red[:,y,:]*data.pixmask[:,y,:]
    X[3,:,:] = data.nir[:,y,:]*data.pixmask[:,y,:]
    X[4,:,:] = data.swir1[:,y,:]*data.pixmask[:,y,:]
    X[5,:,:] = data.swir2[:,y,:]*data.pixmask[:,y,:]
    X[X<=0] = np.nan
    GeoMed[:,y,:] = np.transpose(np.vstack([geometric_median(X[:,:,x],tol,MaxInter) for x in range(0,len(data.x))])) #calculate geomatric median
    Euc = [Eucdistance(GeoMed[:,y,x],X[:,:,x]) for x in range(0,len(data.x))]
    Cos = [Cosdistance(GeoMed[:,y,x],X[:,:,x]) for x in range(0,len(data.x))]
    Eucdist[:,y,:]=np.transpose(np.vstack(Euc))
    cosdist[:,y,:]=np.transpose(np.vstack(Cos))


In [ ]:
def stretch_RGB(data,minimum,maximum):
    
    tmp = data
    #a = (tmp-np.percentile(tmp,3))/(np.percentile(tmp,97)-np.percentile(tmp,3))*255
    img = (tmp-minimum)/(maximum-minimum)*255    
    return img


In [ ]:
from PIL import Image,ImageEnhance
#plot the geometric median with swir2,nir and green band as R,G,B
rgbArray = np.zeros((GeoMed.shape[1],GeoMed.shape[2],3), 'uint8')
rgbArray[..., 0] = (stretch_RGB(GeoMed[5,:,:],100,900))
rgbArray[..., 1] = (stretch_RGB(GeoMed[3,:,:],150,3000))
rgbArray[..., 2] = (stretch_RGB(GeoMed[1,:,:],100,600))
img = Image.fromarray(rgbArray)
plt.imshow(img)

In [ ]:
#plot EMAD and SMAD as the median of EucDist and CosDist

EDist = Eucdist
CDist = cosdist
EMAD = np.nanmedian(EDist,axis=0)
SMAD = np.nanmedian(CDist,axis=0)
plt.imshow(SMAD,cmap='Reds')
plt.colorbar()

In [ ]:
#save Gemetric median to netcdf
ds = xr.Dataset({'blue':(('y','x'),GeoMed[0,:,:]),'green':(('y','x'),GeoMed[1,:,:]),'red':(('y','x'),GeoMed[2,:,:]),\
               'nir':(('y','x'),GeoMed[3,:,:]),'swir1':(('y','x'),GeoMed[4,:,:]),'swir2':(('y','x'),GeoMed[5,:,:]),\
                'EucDist':(('time','y','x'),EDist[:]),'CosDist':(('time','y','x'),CDist[:])},\
               coords={'time':data.time[:],'y':data.y[:],'x':data.x[:]},attrs={'geospatial_bounds_crs':'EPSG:4326','lat_min':data.attrs['lat_min'],\
                                                        'lat_max':data.attrs['lat_max'],'lon_min':data.attrs['lon_min'],\
                                                        'lon_max':data.attrs['lon_max']})
ds.to_netcdf('/g/data/xc0/project/Burn_Mapping/Geometric_Median/Sumac_forest_TAS_GMandDIST_2014_2017_landsat8.nc','w')

In [ ]:
row = 150
col = 100
CDist = cosdist[:,row,col]

In [ ]:
outlier = np.nanpercentile(CDist,75)+1.5*(np.nanpercentile(CDist,75)-np.nanpercentile(CDist,25))
plt.figure(figsize=(8,3))
plt.plot(CDist,'-o')
plt.plot([0,41],[outlier,outlier],'--')
plt.ylabel('CosDist')
plt.xlim(0,41)